## Preparar los datos

In [8]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import plotly.express as px
import plotly.graph_objs as go
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output, State

# Cargar los datos
file_path = './TFM_UNIR/data/processed/BINANCE_BTCUSD_preprocessed.csv'
data = pd.read_csv(file_path)

# Convertir la columna 'time' a datetime
data['time'] = pd.to_datetime(data['time'])

# Ordenar los datos por tiempo
data = data.sort_values('time')

# Seleccionar las características relevantes para la predicción
features = ['open', 'high', 'low', 'close', 'Go_Short', 'Go_Long']
target = 'close'

# Normalizar las características
scaler = MinMaxScaler()
data[features] = scaler.fit_transform(data[features])

# Crear secuencias para el modelo LSTM
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])
    return np.array(X), np.array(y)

seq_length = 10  # Longitud de la secuencia
X, y = create_sequences(data[features].values, seq_length)

# Dividir los datos en conjuntos de entrenamiento y prueba
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Mostrar las formas de los conjuntos de datos
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((14253, 10, 6), (3564, 10, 6), (14253, 6), (3564, 6))

## Crear y entrenar el modelo LSTM

In [11]:
# Crear el modelo LSTM
def build_lstm(input_shape):
    model = Sequential()
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(32))
    model.add(Dense(len(features)))
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Entrenar el modelo LSTM
lstm_model = build_lstm(X_train.shape[1:])
history_lstm = lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

C:\Users\Jhon_\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 17s 15ms/step - loss: 0.0029 - mae: 0.0187 - val_loss: 8.7350e-04 - val_mae: 0.0063
Epoch 2/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 9.7382e-04 - mae: 0.0051 - val_loss: 8.3972e-04 - val_mae: 0.0039
Epoch 3/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0011 - mae: 0.0053 - val_loss: 8.6989e-04 - val_mae: 0.0062
Epoch 4/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 0.0012 - mae: 0.0056 - val_loss: 8.3223e-04 - val_mae: 0.0037
Epoch 5/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 9.8521e-04 - mae: 0.0049 - val_loss: 8.4273e-04 - val_mae: 0.0055
Epoch 6/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.0011 - mae: 0.0049 - val_loss: 8.3116e-04 - val_mae: 0.0034
Epoch 7/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 9.5279e-04 - mae: 0.0045 - val_loss: 8.4628e-04 - val_mae: 0.0055
Epoch 8/10
357/357 ━━━━━━━━━━━━━━━━━━━━ 5s 13ms/step - loss: 9.6170e-04 - mae: 0.0046 - val_loss: 8.4644e-04 - val_mae: 0.0061

## Hacer predicciones

In [14]:
# Hacer predicciones
y_pred = lstm_model.predict(X_test)

# Invertir la normalización para las predicciones y los valores reales
y_test_scaled = scaler.inverse_transform(y_test)
y_pred_scaled = scaler.inverse_transform(y_pred)

# Crear un DataFrame para los valores reales y predichos
time_test = data['time'].values[seq_length + split:]
predictions_df = pd.DataFrame({
    'time': time_test,
    'real': y_test_scaled[:, 3],  # Utilizando la columna 'close'
    'pred': y_pred_scaled[:, 3],  # Utilizando la columna 'close'
    'Go_Short': y_test_scaled[:, 4],  # Utilizando la columna 'Go_Shot'
    'Go_Long': y_test_scaled[:, 5]   # Utilizando la columna 'Go_Long'
})

112/112 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step


In [16]:
# Agregar una columna para las señales combinadas: 1 para compra (Go_Long) y -1 para venta (Go_Short)
predictions_df['signal'] = 0
predictions_df.loc[predictions_df['Go_Short'] == 1, 'signal'] = -1
predictions_df.loc[predictions_df['Go_Long'] == 1, 'signal'] = 1

# Filtrar solo las filas donde hay una señal
signals = predictions_df[predictions_df['signal'] != 0].copy()

# Calcular los retornos diarios
signals['return'] = signals['real'].pct_change()

# Calcular el retorno de la estrategia (suponiendo que se invierte en cada señal)
signals['strategy_return'] = signals['signal'].shift(1) * signals['return']

# Calcular el retorno acumulado
signals['cumulative_return'] = (1 + signals['strategy_return']).cumprod() - 1


## Graficos interactivos

In [19]:
# Crear el dashboard interactivo con Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Predicciones de la Red LSTM y Señales de Trading", style={'textAlign': 'center'}),

    # Casillas de selección para mostrar u ocultar señales
    html.Label("Seleccione las Señales:"),
    dcc.Checklist(
        id='signal-checklist',
        options=[
            {'label': 'Go_Short', 'value': 'Go_Short'},
            {'label': 'Go_Long', 'value': 'Go_Long'}
        ],
        value=['Go_Short', 'Go_Long'],  # Valores por defecto
        inline=True
    ),
    # Selector de tipo de moneda y monto de inversión
    html.Label("Seleccione el tipo de moneda:"),
    dcc.RadioItems(
        id='currency-selector',
        options=[
            {'label': 'BTC', 'value': 'BTC'},
            {'label': 'USD', 'value': 'USD'}
        ],
        value='BTC',  # Valor por defecto
        inline=True
    ),
    
    html.Label("Monto de Inversión:"),
    dcc.Input(id='investment-amount', type='number', value=1, min=0.1, step=0.1),

    # Botón para actualizar la gráfica
    html.Button('Actualizar Gráfica', id='update-button', n_clicks=0),

    # Filas de gráficos
    html.Div([
        dcc.Graph(id='prediction-graph', style={'display': 'inline-block', 'width': '49%'}),
        dcc.Graph(id='portfolio-graph', style={'display': 'inline-block', 'width': '49%'}),
    ], style={'display': 'flex', 'flex-direction': 'row'}),
    
    # Fila de tabla de métricas
    html.Div([
        dash_table.DataTable(
            id='metrics-table',
            columns=[
                {"name": "Métrica", "id": "metric"},
                {"name": "Valor", "id": "value"}
            ],
            style_data={
                'whiteSpace': 'normal',
                'height': 'auto',
            },
            style_cell={
                'textAlign': 'left',
                'padding': '5px'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'column_id': 'value'},
                    'backgroundColor': 'rgb(248, 248, 248)',
                    'color': 'black'
                }
            ],
            style_table={'width': '100%'}
        )
    ])
])

@app.callback(
    [
        Output('prediction-graph', 'figure'),
        Output('metrics-table', 'data'),
        Output('portfolio-graph', 'figure')
    ],
    [
        Input('signal-checklist', 'value'),
        Input('currency-selector', 'value'),
        Input('update-button', 'n_clicks')
    ],
    [State('investment-amount', 'value')]
)

def update_graph(selected_signals, currency, n_clicks, investment_amount):
    fig = go.Figure()
    
    # Gráfico de Predicciones: Asegúrate de usar todos los datos
    fig.add_trace(go.Scatter(x=predictions_df['time'], y=predictions_df['real'],
                             mode='lines', name='Valor Real'))
    fig.add_trace(go.Scatter(x=predictions_df['time'], y=predictions_df['pred'],
                             mode='lines', name='Predicción LSTM'))

    # Filtrar las señales solo cuando sean 1
    go_short_signals = predictions_df[predictions_df['Go_Short'] == 1]
    go_long_signals = predictions_df[predictions_df['Go_Long'] == 1]

    # Gráfico de Señales Go_Short
    if 'Go_Short' in selected_signals:
        fig.add_trace(go.Scatter(x=go_short_signals['time'], y=go_short_signals['real'],
                                 mode='markers', marker_symbol='triangle-down', name='Go_Short',
                                 marker=dict(size=10, color='red')))

    # Gráfico de Señales Go_Long
    if 'Go_Long' in selected_signals:
        fig.add_trace(go.Scatter(x=go_long_signals['time'], y=go_long_signals['real'],
                                 mode='markers', marker_symbol='triangle-up', name='Go_Long',
                                 marker=dict(size=10, color='green')))

    fig.update_layout(title='Predicciones de la Red LSTM y Señales de Trading',
                      xaxis_title='Tiempo', yaxis_title='Valor')

    # Calcular métricas
    metrics = calculate_metrics(signals)

    metrics_data = [
        {"metric": "Retorno Total", "value": f"{metrics['total_return']:.2f}"},
        {"metric": "Tasa de Sharpe", "value": f"{metrics['sharpe_ratio']:.2f}"},
        {"metric": "Drawdown Máximo", "value": f"{metrics['max_drawdown']:.2f}"},
        {"metric": "Número de Operaciones", "value": f"{metrics['number_of_trades']}"},
        {"metric": "Precisión", "value": f"{metrics['accuracy']:.2f}"},
        {"metric": "Ratio Ganancia/Pérdida", "value": f"{metrics['win_loss_ratio']:.2f}"},
        {"metric": "Ratio de Sortino", "value": f"{metrics['sortino_ratio']:.2f}"}
    ]
    
    # Calcular el patrimonio a lo largo del tiempo según el tipo de moneda y el monto de inversión
    if currency == 'USD':
        # Suponiendo que el precio inicial es el precio de cierre del primer valor de la serie de prueba
        initial_price = y_test_scaled[0][3]
        initial_investment_btc = investment_amount / initial_price
    else:
        initial_investment_btc = investment_amount

    signals['portfolio_value'] = initial_investment_btc * (1 + signals['cumulative_return'])

    # Gráfico de Patrimonio: Asegúrate de usar todos los datos
    portfolio_fig = go.Figure()
    portfolio_fig.add_trace(go.Scatter(
        x=signals['time'], 
        y=signals['portfolio_value'], 
        mode='markers', 
        name='Valor del Patrimonio'
    ))

    portfolio_fig.update_layout(
        title=f'Valor del Patrimonio al Invertir {investment_amount} {currency}',
        xaxis_title='Tiempo',
        yaxis_title='Valor del Patrimonio (BTC)'
    )

    return fig, metrics_data, portfolio_fig

def calculate_metrics(signals):
    total_return = signals['cumulative_return'].iloc[-1]
    sharpe_ratio = signals['strategy_return'].mean() / signals['strategy_return'].std() * np.sqrt(252)
    cumulative_max = signals['cumulative_return'].cummax()
    drawdown = (signals['cumulative_return'] - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()
    number_of_trades = len(signals)
    winning_trades = signals[signals['strategy_return'] > 0]
    accuracy = len(winning_trades) / number_of_trades
    average_win = winning_trades['strategy_return'].mean()
    losing_trades = signals[signals['strategy_return'] <= 0]
    average_loss = losing_trades['strategy_return'].mean()
    win_loss_ratio = average_win / abs(average_loss) if average_loss != 0 else np.nan
    downside_risk = signals[signals['strategy_return'] < 0]['strategy_return'].std()
    sortino_ratio = signals['strategy_return'].mean() / downside_risk * np.sqrt(252)

    return {
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'number_of_trades': number_of_trades,
        'accuracy': accuracy,
        'win_loss_ratio': win_loss_ratio,
        'sortino_ratio': sortino_ratio
    }
if __name__ == '__main__':
    app.run_server(debug=True, port=8054)  # Cambiar el puerto aquí si es necesario
